<a href="https://colab.research.google.com/github/mdxsuhail/JEE-kcet-comdk-rank-predictor/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import warnings

# --- 1. CONFIGURATION ---
warnings.filterwarnings("ignore") # Clean output

# --- 2. USER INPUTS (Change these to test!) ---
# Since GitHub can't show buttons, we set the values here.
MY_EXAM = 'COMEDK'   # Options: 'KCET', 'COMEDK', 'JEE'
MY_MARKS = 115       # Enter your marks here

# --- 3. THE MASTER DATABASE (2025 Official Data) ---
master_data = [
    # TIER 1
    { "College": "RV College of Engineering", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 542, "KCET_Rank": 250 },
    { "College": "RV College of Engineering", "Branch": "Electronics (ECE)", "COMEDK_Rank": 1077, "KCET_Rank": 900 },
    { "College": "MS Ramaiah (MSRIT)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 1341, "KCET_Rank": 900 },
    { "College": "BMS College of Engineering", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 3689, "KCET_Rank": 1000 },
    { "College": "BMS College of Engineering", "Branch": "Electronics (ECE)", "COMEDK_Rank": 5302, "KCET_Rank": 3500 },

    # TIER 2
    { "College": "Dayananda Sagar (DSCE)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 5479, "KCET_Rank": 4500 },
    { "College": "Bangalore Inst. of Technology", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 9177, "KCET_Rank": 6000 },
    { "College": "CMR Institute of Tech (CMRIT)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 18084, "KCET_Rank": 14000 },
    { "College": "Sir M Visvesvaraya (Sir MVIT)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 22026, "KCET_Rank": 15000 },

    # TIER 3 & REGIONAL
    { "College": "KLE Tech (Hubballi)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 31388, "KCET_Rank": 13000 },
    { "College": "Dayananda Sagar Academy (DSATM)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 35187, "KCET_Rank": 20000 },
    { "College": "Acharya Institute (AIT)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 51728, "KCET_Rank": 25000 },
    { "College": "SJB Institute of Tech", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 53748, "KCET_Rank": 28000 }
]

df = pd.DataFrame(master_data)
df['JEE_Rank'] = 999999 # Placeholder

# --- 4. TRAIN AI MODELS (Marks -> Rank) ---
kcet_train = pd.DataFrame({'Marks': [175, 170, 160, 150, 140, 130, 120, 110, 100, 90, 80, 70, 60], 'Rank': [10, 50, 200, 500, 1500, 3500, 6000, 10000, 18000, 30000, 45000, 65000, 90000]})
comedk_train = pd.DataFrame({'Marks': [170, 160, 150, 140, 130, 120, 110, 100, 90, 80, 70, 60], 'Rank': [15, 100, 400, 1200, 2500, 4500, 7500, 12000, 19000, 28000, 40000, 55000]})
jee_train = pd.DataFrame({'Marks': [280, 260, 240, 220, 200, 180, 160, 140, 120, 100], 'Rank': [50, 500, 1500, 3500, 6500, 12000, 22000, 38000, 65000, 110000]})

kcet_model = make_pipeline(PolynomialFeatures(3), LinearRegression())
kcet_model.fit(kcet_train[['Marks']], kcet_train['Rank'])

comedk_model = make_pipeline(PolynomialFeatures(3), LinearRegression())
comedk_model.fit(comedk_train[['Marks']], comedk_train['Rank'])

jee_model = make_pipeline(PolynomialFeatures(3), LinearRegression())
jee_model.fit(jee_train[['Marks']], jee_train['Rank'])

# --- 5. PREDICTION LOGIC ---
def predict_and_display(exam, marks):
    input_data = pd.DataFrame({'Marks': [marks]})

    if exam == 'KCET':
        pred_rank = int(kcet_model.predict(input_data)[0])
    elif exam == 'COMEDK':
        pred_rank = int(comedk_model.predict(input_data)[0])
    else:
        pred_rank = int(jee_model.predict(input_data)[0])

    if pred_rank < 1: pred_rank = 1

    col_name = f"{exam}_Rank"
    candidates = df[df[col_name] < 999999].copy()
    eligible = candidates[candidates[col_name] >= pred_rank].sort_values(by=col_name)

    def get_status(cutoff, my_rank):
        if my_rank < (cutoff * 0.8): return "🟢 Safe"
        if my_rank < cutoff: return "🟡 Likely"
        return "🔴 Borderline"

    eligible['Chances'] = eligible[col_name].apply(lambda x: get_status(x, pred_rank))

    # --- DISPLAY REPORT ---
    print(f"🤖 AI PREDICTION REPORT")
    print(f"=======================")
    print(f"📌 Exam: {exam}")
    print(f"📌 Marks: {marks}")
    print(f"📊 Predicted Rank: {pred_rank:,}")
    print(f"-----------------------")

    if eligible.empty:
        print("⚠️ No colleges found for this rank in the current database.")
    else:
        print(f"✅ Top Recommendations:")
        # We use print instead of display() for better GitHub compatibility
        print(eligible[['College', 'Branch', col_name, 'Chances']].head(10).to_string(index=False))

# --- 6. RUN THE PREDICTION ---
predict_and_display(MY_EXAM, MY_MARKS)


🤖 AI PREDICTION REPORT
📌 Exam: COMEDK
📌 Marks: 115
📊 Predicted Rank: 5,634
-----------------------
✅ Top Recommendations:
                        College                 Branch  COMEDK_Rank Chances
  Bangalore Inst. of Technology Computer Science (CSE)         9177  🟢 Safe
  CMR Institute of Tech (CMRIT) Computer Science (CSE)        18084  🟢 Safe
  Sir M Visvesvaraya (Sir MVIT) Computer Science (CSE)        22026  🟢 Safe
            KLE Tech (Hubballi) Computer Science (CSE)        31388  🟢 Safe
Dayananda Sagar Academy (DSATM) Computer Science (CSE)        35187  🟢 Safe
        Acharya Institute (AIT) Computer Science (CSE)        51728  🟢 Safe
          SJB Institute of Tech Computer Science (CSE)        53748  🟢 Safe
